# RQTR (for Lemmatized Corpora)

With this notebook you can calculate the RQTR(n) values for a lemmatized corpus.

In [ ]:
# SETUP

from src.corpus import Corpus
from src.metrics import rqtr_lemma
from src.corpus_creation import document_retriever
import pandas as pd
from src.load_data import load_files

%load_ext autoreload
%autoreload 2

Loading the data...

Put the path to your corpus in the variable `CORPUSDIR`.

I assume that the data is a set of json files, each containing a list of lemmata under the key 'lemmas'.
If you have a different format, you need to adjust the code accordingly. The result should be a list of lists of lemmata.

In [ ]:
# Put the path to the directory containing the corpus files here
CORPUSDIR = '/home/brunobrocai/Data/MoWiKo/Paper-themKorp/full'

docs, metadata = load_files(CORPUSDIR)
corpus = Corpus(docs, metadata)

Picking the ANY NUMBER OF (!) base terms for RQTR calculation...

In [ ]:
# Picking base terms
base_terms = (('künstlich', 'Intelligenz'), 'KI', 'AI')

Here it comes!

Let's calculcate baseline QTR values.

In [ ]:
b, core_term =rqtr_lemma.qtr_baseline(
    base_terms, corpus
)

Now we can calculate RQTR values for all terms in the corpus -- at least those that cooccur with the base terms at least once.

In [ ]:
cooccurence_values = rqtr_lemma.count_cooccurence(
    base_terms,
    corpus,
    max_ngram_len=1,
)

The function we just used returns a dictionary with the RQTR values for all terms in the corpus. We can now perform some Pandas DataFrame magic to get a nice overview of the results.

In [ ]:
rqtrn_table = rqtr_lemma.cooccurence_to_metric(
    cooccurence_values,
    b,
    metric='rqtrn'
)
rqtrn_table

In [ ]:
# Get the values with RQTRN > 40
filtered_df = rqtrn_table[(rqtrn_table['RQTRN'] > 40)]
# Remove the values with RQTRN == 100 because they are not useful
filtered_df = filtered_df[(filtered_df['RQTRN'] < 100)]

filtered_df

## Part 2: Corpus Creation
Now we can retrieve documents based on the wordlist we created with the RQTR method.

In [ ]:
from src.corpus_creation import document_retriever as dr


wordlist = filtered_df['Term'].tolist()
wordlist.extend(base_terms)
hits = dr.match_wordlist(corpus, wordlist, min=1)

We can create new annotation files with the prepare_annotations() function below.
Pass a key such as 'wird_annotiert' as annotator to skip annotations that are already present in the file (or, in the case of 'wird_annotiert', will soon be annotated).

In [ ]:
dr.prepare_annotations(
    corpus,
    hits,
    annotator='wird_annotiert',  # Die müssen wir nicht nochmal annotieren
    goalpath='test_prep_anno'
)